In [2]:
from collections import deque
import pickle
from random import choice

def select_root_with_minimum_m(G, k):
    m_values = {}
    
    for v in range(len(G)):
        m_v = 0
        while len(get_N_h(G, v, m_v)) < k:
            m_v += 1
        m_values[v] = m_v
    return min(m_values, key=m_values.get)


def children_of(G, v, visited):
    return [node for node in G[v] if node not in visited]

def labeled_neighbour_of(G, v, visited, k):
    m_v = 0
    while len(get_N_h(G, v, m_v)) < k:
        m_v += 1
    neighbour = list(get_N_h(G, v, 2 * m_v))
    return [node for node in neighbour if node in visited]


def get_N_h(G, v, h):
    visited = set()
    queue = deque([(v, 0)])

    while queue:
        node, depth = queue.popleft()
        if node not in visited:
            visited.add(node)
            if depth < h:
                for neighbor in G[node]:
                    if neighbor not in visited:
                        queue.append((neighbor, depth + 1))
    return visited

def r_m_values(G, labels_assigned, k):
    r_values = {}
    m_values = {}

    for v in range(len(G)):
        r_v = 0
        m_v = 0
        while len(set(labels_assigned[u] for u in get_N_h(G, v, r_v))) < k:
            r_v += 1
        r_values[v] = r_v

        while len(get_N_h(G, v, m_v)) < k:
            m_v += 1
        m_values[v] = m_v

    return r_values, m_values


def tree_node_labelling(G, k, d=2):
    def label_impact(v, label):
        neighbors = get_N_h(G, v, d)
        impact = sum(1 for u in neighbors if labels_assigned.get(u, -1) == label)
        return impact

    K = list(range(k))
    r = select_root_with_minimum_m(G, k)
    BFS_queue = deque([(r, [])])
    labels_assigned = {}
    labels_used = [False] * len(K)

    while BFS_queue:
        v, visited = BFS_queue.popleft()

        if all(labels_used):
            labels_used = [False] * len(K)

        if v != r:
            labeled_neighbour = labeled_neighbour_of(G, v, visited, k)
            available_labels = [label for i, label in enumerate(K) if not labels_used[i] and label not in [labels_assigned[u] for u in labeled_neighbour]]
        else:
            available_labels = [label for i, label in enumerate(K) if not labels_used[i]]

        # In case available_labels is empty after resetting labels_used
        if not available_labels:
            available_labels = [label for i, label in enumerate(K) if not labels_used[i]]

        best_label = min(available_labels, key=lambda x: label_impact(v, x))
        labels_assigned[v] = best_label
        labels_used[K.index(best_label)] = True

        visited.append(v)
        for child in children_of(G, v, visited):
            BFS_queue.append((child, visited.copy()))

    return labels_assigned


# Load input instances

trees = pickle.load(open("./Examples_of_AdjLists_of_Trees", "rb"))
k_values = pickle.load(open("./Examples_of_k_values", "rb"))


"""
trees = pickle.load(open("./Small_Examples_of_AdjLists_of_Trees", "rb"))
k_values = pickle.load(open("./Small_Examples_of_k_values", "rb"))
"""
"""
trees = pickle.load(open("./Medium_Examples_of_AdjLists_of_Trees", "rb"))
k_values = pickle.load(open("./Medium_Examples_of_k_values", "rb"))
"""


# trees = pickle.load(open("./Medium_Examples_of_AdjLists_of_Trees", "rb"))
# k_values = pickle.load(open("./Medium_Examples_of_k_values", "rb"))


# Compute solutions
solutions = []

# Compute proximity ratio
max_ratios = []

for i in range(len(trees)):
    G = trees[i]
    k = k_values[i]
    result = tree_node_labelling(G, k,)
    solutions.append(list(result.values()))

    r_values, m_values = r_m_values(G, result, k)
    ratios = [r_values[v] / m_values[v] for v in range(len(G))]
    max_ratios.append(max(ratios))
    
# Save solutions
pickle.dump(solutions, open("solutions", "wb"))

print(max_ratios)
print(max(max_ratios))
print(solutions)

[1.5, 2.0, 1.5, 1.5, 1.5, 1.3333333333333333, 1.6666666666666667, 1.3333333333333333, 1.0, 1.0]
2.0
[[0, 1, 2, 3, 4, 5, 1, 2, 4, 5, 3, 0], [0, 1, 2, 2, 0, 1, 2, 0, 1, 2, 0, 1], [0, 1, 2, 3, 4, 5, 6, 2, 1, 3, 4, 5], [0, 1, 2, 3, 4, 5, 6, 1, 3, 4, 5, 0], [0, 1, 2, 3, 1, 2, 0, 3, 1, 3, 0, 2], [0, 1, 2, 3, 4, 5, 6, 1, 2, 3, 5], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 4], [0, 1, 2, 3, 4, 5, 6, 7, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 3], [0, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 4]]
